In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][1]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.log',
    'search_read',
    [],
    {'fields': [] })

id_field_into_1value(df)

df = pd.DataFrame(df)

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
df.to_sql('odoo_log', con=engine, if_exists='replace', index=False)

755